# Supervised Learning

Let's implement least means square fitting manually, and check to see how different parameters affect learning

In [ ]:
# Required imports
import numpy as np
import matplotlib.pyplot as plt

## 0) OLS
Taken from https://github.com/jermwatt/machine_learning_refined

In [ ]:
# import the dataset
csvname =  'student_debt_data.csv'
data = np.loadtxt(csvname,delimiter=',')

# extract input - for this dataset, these are times
x = data[:,0]

# extract output - for this dataset, these are total student debt
y = data[:,1]

print(np.shape(x))
print(np.shape(y))


## 1) LMS
### Create a pseudo-random dataset

Given the tensor `x` in space $\mathbb{R}^{\text{j}}$ with `i` samples and `j` features, generate `y` such that it holds some correlation to `x`.

A *tensor* is simple a generalized matrix that can be any dimension, not just 2D. We will stick with 2D here for simplicity.

In [ ]:
# Set parameters
J = 5 # num features
I = 30 # num samples in dataset
NOISE_AMPLITUDE = 0.1 # how much noise to inject into dataset

# Generate x values
x = np.random.uniform(-1, 1, (I,J))
print(f'Shape of array x is {x.shape}') # check the shape

# Generate a y value that is somewhat correlated
# There are many ways to do this, let's just write a quick rule that linearly combines the features of x and injects some noise
# Note that we assume an intercept (bias) of 0
true_weights = np.random.uniform(-1,1, (J))

# WHAT IS GOIG ON HERE?
y = x @ true_weights + np.random.normal(0,NOISE_AMPLITUDE,(I)) # the `@` operator is used for numpy matrix multiplication, and is just shorthand for the dot product

# WHAT SHOULD THE CORR BE?
# Let's check the correlation to be sure
correlations = [np.corrcoef(x[:, i], y)[0, 1] for i in range(J)] # calculate Pearson's R for each feature vector
print(f'Correlations for each feature are {correlations}')

Chances are, some features will have a large correlation whereas others may have little correlation. Note that correlation here is only checking for the existence of a *linear* relationship.

If we wanted to improve our correlations, is it better to add more data or to reduce the amplitude of noise? Come up with a hypothesis and test it by tuning the parameters above.

Experiment with the number of samples, features, and amplitude of noise.

### Visualize dataset

In [ ]:
plt.figure(figsize=(20, 1*J))

colors = plt.cm.Set1(np.linspace(0, 1, J)) # make an iterable of colors

for feature in range(J):
    plt.subplot(1, J, feature+1)
    plt.scatter(x[:,feature],y,color = colors[feature])
    plt.title(f"Feature {feature+1}")
    plt.xlabel(f"$x_{feature+1}$")
    plt.ylabel("y")

In [ ]:
# for ease of use later, let's wrap up our dataset generation into a function that gives us an x and y given our starting params

def generate_dataset(num_feats: int, num_samples: int, noise_amp: float) -> tuple([np.ndarray, np.ndarray, np.ndarray]):
    x = np.random.uniform(-1, 1, (num_samples, num_feats))
    true_weights = np.random.uniform(-1,1, (num_feats))
    y = x @ true_weights + np.random.normal(0, noise_amp, (num_samples))
    return x, y, true_weights

### Least Mean Squares

We will now implement LMS to see if we can recover our true weights!

We can start by taking a random guess of our true weights. Let's call our random starting weights `θ`

In [ ]:
theta = np.random.uniform(-2,2,(J)) # here, we are initializing our weights from a uniform distribution ranging from -2 to 2, but it could start from anywhere
print(f'theta is {theta}')

Now, we can apply those weights `θ` (our hypothesis) to our data `x` to calculate our predicted y, or $\hat{y}$

In [ ]:
yhat = x @ theta

Then, define the LMS cost function. Here, our cost function we would like to minimize is the *mean squared error* between y and yhat. Let's write a function so that this is easy to compute. We will use good coding practice here as well by properly documenting our function with typings and naming our variables to be easily understood.

In [ ]:
def mean_squared_error(x : np.ndarray, y : np.ndarray, theta : np.ndarray) -> np.ndarray:
    yhat = x @ theta # apply weights to data to get prediction yhat
    error = yhat - y # get the error
    loss = (1 / len(y)) * np.sum(error ** 2) #mean squared error
    return loss

# test it out
print(f'Initial loss: {mean_squared_error(x,y,theta)}')

Now we will do gradient descent. This is an iterative algorithm that requires us to take the partial derivative of our loss function, with respect to our parameters θ. Below is the derivation provided again for review.

---

#### __Cost Function__:

The cost function for LMS is defined as the Mean Squared Error (MSE):

$$ J(\theta) = \frac{1}{2m} \sum_{i=1}^{m} (h_{\theta}(x^{(i)}) - y^{(i)})^2 $$
Where:
- $m$ is the number of data points.
- $h_{\theta}(x^{(i)})$ is our hypothesis or prediction for the $i^{th}$ input. 

---

#### __Hypothesis__:

Our hypothesis function is given by:

$$ h_{\theta}(x) = x \cdot \theta $$
Where:
- $x$ is our input vector.
- $\theta$ is our weight vector.

---

#### __Gradient of the Cost Function__:

To perform gradient descent, we need the gradient of the cost function with respect to our weights, $\theta$. The gradient will tell us how to update $\theta$ to reduce our cost.

For each weight $\theta_j$, the partial derivative is:

$$ \frac{\partial}{\partial \theta_j} J(\theta) = \frac{1}{m} \sum_{i=1}^{m} (h_{\theta}(x^{(i)}) - y^{(i)}) x_j^{(i)} $$

#### __Gradient Descent Update Rule__:

Given the above gradient, the update rule for gradient descent is:

$$ \theta_j := \theta_j - \alpha \frac{\partial}{\partial \theta_j} J(\theta) $$
Where:
- $\alpha$ is our __learning rate__.


Substituting in our derived gradient:

$$ \theta_j := \theta_j - \alpha \frac{1}{m} \sum_{i=1}^{m} (h_{\theta}(x^{(i)}) - y^{(i)}) x_j^{(i)} $$

This update rule is applied simultaneously for all values of $j$.

$:=$ simply means the left side of the equation is updated by the right side of the equation (not the other way around)


---


#### __Derivation of the Gradient__:

Now, for the step-by-step derivation of the gradient:

1. Start with a single example $i$ inside the summation:

$$ \text{error}^{i} = h_{\theta}(x^{i}) - y^{i} $$

2. Squared error for the $i^{th}$ example:

$$ (\text{error}^{i})^2 = (x^{i} \cdot \theta - y^{i})^2 $$

3. Take the partial derivative with respect to $\theta_j$:

$$ \frac{\partial}{\partial \theta_j} (\text{error}^{i})^2 = 2 (x^{i} \cdot \theta - y^{i}) x_j^{i} $$

4. The above result gives us the gradient of a single squared error term. To get the gradient for the MSE, we need to average over all $m$ training examples and account for the $\frac{1}{2}$ term in front (which conveneniently reduces to 1):

$$ \frac{\partial}{\partial \theta_j} J(\theta) = \frac{1}{m} \sum_{i=1}^{m} (h_{\theta}(x^{(i)}) - y^{(i)}) x_j^{(i)} $$

And that gives us the gradient for mean squared error!

Let's define that function in python.

In [ ]:
def calculate_gradient_and_update(x: np.ndarray, y: np.ndarray, theta: np.ndarray, alpha: float) -> tuple([float, np.ndarray]):
    gradient = (1 / len(y)) * x.T @ ((x @ theta) - y) # use the above formula to calculate the gradient of the loss
    theta_new = theta - (alpha * gradient) # update the parameters according to our update function
    loss = mean_squared_error(x, y, theta_new) # find the new loss
    return loss, theta_new

# test it out
loss, _ = calculate_gradient_and_update(x, y, theta, 0.01)
print(loss)

Now, let's put it all together and perform *batch gradient descent*

Batch gradient descent entails calculating the loss over the entire training dataset. This is only possible if your dataset can fit into memory, but guarantees convergence.

In [ ]:
# first, let's regenerate our dataset for convenience
x, y, true_weights = generate_dataset(J, I, 1)

In [ ]:
loss_history = [] #track our losses as we update
num_iterations = 100 #how many times we update our parameters
ALPHA = 0.1 #learning rate, this value is usually way smaller for nonlinear models (like 1e-4)

theta = np.random.uniform(-2,2,(J)) #initialize our starting weights

for t in range(num_iterations):
    loss, theta = calculate_gradient_and_update(x, y, theta, ALPHA)
    loss_history.append(loss)

print(f'Final loss: {loss}')

# plot the losses over time
plt.figure(0)
plt.plot(loss_history);
plt.title('Training loss');
plt.xlabel('Iterations');
plt.ylabel('Loss');

#did we recover the true weights?
plt.figure(1)
plt.scatter(range(J),true_weights, color = 'black')
plt.scatter(range(J), theta, color = 'red', marker = 'x')
plt.legend(['True weights','Learned weights'])

### Problem 1: Noise

Our loss is plateauing at some value. Why doesn't it drop to zero? What is the significance of this value? Why can't we recover the true weights? 

Let's do a little experiment...

In [ ]:
# How does our noise in our dataset relate to our final loss? Well, we can easily plot them against each other so let's start there
noises = np.arange(0,2,0.01)
num_iterations = 10
learning_rate = 1 #set this to a unreasonably large number because we are optimizing a convex function and want to converge fast
final_losses = []

for n in noises:
    x,y,true_weights = generate_dataset(J, I, n)
    theta = np.random.uniform(-2,2,(J))
    for i in range(num_iterations):
        loss, theta = calculate_gradient_and_update(x, y, theta, 1)
    final_losses.append(loss)

#now plot our final losses against our dataset noise
plt.scatter(noises, final_losses)
plt.title('Influence of noise on loss')
plt.xlabel('Noise amplitude')
plt.ylabel('Final loss')

Work through these problems and discuss your answers.

1. Derive the relationship you see here, given what you know about our loss function.
2. How does the distribution of our training data and weights relate to the amplitude of noise? What common engineering concept can be applied to describe this?
3. How do you know your model has learned to approximate some real world function, and not just noise?

(fyi, this type of noise in our data is called *aleatoric uncertainty* because it cannot be reduced by collecting more data)

### Problem 2. Batch vs Stochastic Gradient Descent

Batch gradient descent requires computation of the gradient for every sample of your training data, before updating your parameters. However, this can be computationally inefficient. Stochastic gradient descent (SGD) entails learning on each sample, one at a time. Your model will learn much faster, but runs the risk of destabilizing due to noisy inputs.

Let's adapt the above code to do SGD on our dataset, and observe our loss.

In [ ]:
# first, let's regenerate our dataset for convenience
x, y, true_weights = generate_dataset(J, I, 1)

loss_history = []
num_epochs = 10 # how many times we iterate through our whole dataset, element by element
ALPHA = 0.1 

theta = np.random.uniform(-2,2,(J)) # initialize our starting weights

for e in range(num_epochs):
    for i in range(len(x)):
        loss, theta = calculate_gradient_and_update(np.expand_dims(x[i,:],0), np.expand_dims(y[i],0), theta, ALPHA) # we update the size 
        loss_history.append(loss)

print(f'Final loss: {loss}')

# plot the losses over time
plt.figure(0)
plt.plot(loss_history);
plt.title('Training loss');
plt.xlabel('Iterations');
plt.ylabel('Loss');

#did we recover the true weights?
plt.figure(1)
plt.scatter(range(J),true_weights, color = 'black')
plt.scatter(range(J), theta, color = 'red', marker = 'x')
plt.legend(['True weights','Learned weights'])

Why do you see a repeated pattern in the loss curve? Why is it harder to converge to the true weights?

### Problem 3. Visualize learning

A visual example of how batch gradient descent and stochastic gradient descent differ can be useful to see. Let's regenerate our dataset, now with just 2 features.

In [ ]:
#make our dataset
x, y, true_weights = generate_dataset(2, I, 1)
starting_theta = np.random.uniform(-2,2,(2)) #initialize our starting weights

Let's train on this new dataset using both methods. Make sure to start from the same initial parameters.

In [ ]:
#perform batch gradient descent
theta_history_batch = [] #track our parameters as we update them
num_iterations = 100 #how many times we update our parameters
ALPHA = 0.1 #learning rate, this value is usually way smaller for nonlinear models (like 1e-4)

theta = starting_theta
for t in range(num_iterations):
    _, theta = calculate_gradient_and_update(x, y, theta, ALPHA)
    theta_history_batch.append(theta)

theta_history_batch = np.array(theta_history_batch)

In [ ]:
#repeat now with SGD
theta_history_sgd = []
num_epochs = 10 #how many times we iterate through our whole dataset, element by element
ALPHA = 0.1 

theta = starting_theta
for e in range(num_epochs):
    for i in range(len(x)):
        _, theta = calculate_gradient_and_update(np.expand_dims(x[i,:],0), np.expand_dims(y[i],0), theta, ALPHA) #we update the size 
        theta_history_sgd.append(theta)

theta_history_sgd = np.array(theta_history_sgd)

We can plot the trajectory of our parameters as it travels over the loss landscape.

In [ ]:
#get the loss landscape
param_space = np.linspace(-2,2,100) #define our grid of points to evaluate
theta0, theta1 = np.meshgrid(param_space, param_space)
loss_scape = []

for t0, t1 in zip(np.ravel(theta0), np.ravel(theta1)): #for every coordinate in our grid
    l, _ = calculate_gradient_and_update(x,y,np.array([t0,t1]),1) #calculate loss
    loss_scape.append(l)
loss_scape = np.array(loss_scape).reshape(theta0.shape) #get it back in proper shape

#plot the loss landscape and each training method
plt.figure(0, figsize = (12,12))
plt.title('Parameter Optimization', fontsize = 20)
plt.contourf(theta0, theta1, loss_scape, levels=np.logspace(-2, 3, 100), cmap = 'inferno')
plt.clim(np.min(loss_scape),np.max(loss_scape))
plt.colorbar()
plt.xlabel('theta_0')
plt.ylabel('theta_1')
plt.plot(theta_history_sgd[:,0],theta_history_sgd[:,1], color = 'white')
plt.plot(theta_history_batch[:,0],theta_history_batch[:,1], color = 'cyan')

plt.legend(['SGD','Batch'])

Talk through and code up answers to the following questions.

1. Implement a learning rate scheduler for stochastic gradient descent.
2. How do you make a scheduler sensitive to learning in real time? Think of the trajectory in physics terms.

### (BONUS) Problem 4: Gradient descent with a nonlinear function

Real world data is almost always nonlinear. Though we have not yet covered nonlinear models, it is frequent practice to apply a linear model to nonlinear datasets. Let's use a classic function, Himmelblau's function.

$$f(x, y) = (x^2 + y - 11)^2 + (x + y^2 - 7)^2$$

It is commonly used for testing optimization algorithms because it has multiple local minima. To perform gradient descent directly on this function, you need to find the partial derivatives of $f(x, y)$ in terms of both $x$ and $y$. To be clear, there is no cost function to plug in here, we are using the derivative of the Himmelblau function directly as our cost function or **objective function** and minimizing it!

Try deriving these yourself and plugging them in to the code below. Then, the code below shows how gradient descent can result in very different solutions, depending on initial conditions.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#try updating this and seeing how the trajectory changes
STARTING_COORDS = (0,0)

def himmelblau(x, y):
    return (x**2 + y - 11)**2 + (x + y**2 - 7)**2

def himmelblau_gradient_descent(lr=0.01, n_iterations=1000, init_point=(-4, 4)):
    x, y = init_point
    history = [(x, y)]
    
    for _ in range(n_iterations):
        gradient_x = # put solution here
        gradient_y = # put solution here
        
        x -= lr * gradient_x
        y -= lr * gradient_y
        
        history.append((x, y))
        
    return np.array(history)

history = himmelblau_gradient_descent(lr=0.01, n_iterations=100, init_point=(0,0)) #why does increasing learning rate throw an error?

# Visualizing the optimization path
x = np.linspace(-6, 6, 400)
y = np.linspace(-6, 6, 400)
X, Y = np.meshgrid(x, y)
Z = himmelblau(X, Y)

plt.figure(figsize=(10, 8))
plt.contourf(X, Y, Z, 50, cmap='viridis')
plt.colorbar()
plt.plot(history[:, 0], history[:, 1], c='red')  # Plotting the path
plt.title("Gradient Descent on Himmelblau's Function")
plt.xlabel('x')
plt.ylabel('y')
plt.show()

#### Answer to bonus

Try deriving this yourself first.

        gradient_x = 4*x*(x**2 + y - 11) + 2*(x + y**2 - 7)
        gradient_y = 2*(x**2 + y - 11) + 4*y*(x + y**2 - 7)